In [48]:
try:
    print('Downloading needed packages...')
    !conda install -c conda-forge geocoder --yes
    !conda install -c conda-forge folium --yes
    print('Success: All lib downloaded')
except:
    print('Error')

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import types
from botocore.client import Config
import ibm_boto3

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Success: All lib downloaded
Libraries imported.


In [15]:
df_areas = pd.read_csv("toronto_canada.csv")
borough_area = df_areas.index[df_areas['Borough'] == 'Not assigned']
df_areas.drop(df_areas.index[borough_area], inplace=True)

neighbourhood_area = df_areas.index[df_areas['Neighbourhood'] == 'Not assigned']
df_areas.drop(df_areas.index[neighbourhood_area], inplace=True)

df_areas.reset_index(drop=True, inplace=True)
df_areas.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [43]:
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")

In [44]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df_areas['Borough'].unique()),
        df_areas.shape[0]
    )
)

The dataframe has 11 boroughs and 200 neighbourhoods.


In [46]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [47]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_areas['Borough'], df_areas['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [52]:
north_york_data = df_areas[df_areas['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M6A,North York,Lawrence Heights
3,M6A,North York,Lawrence Manor
4,M3B,North York,Don Mills North


In [53]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [54]:
# create map of Toronto using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], north_york_data['Borough'], north_york_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

In [55]:
CLIENT_ID = 'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L' # your Foursquare ID
CLIENT_SECRET = '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L
CLIENT_SECRET:1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33


In [56]:
north_york_data.loc[0, 'Neighbourhood']

'Parkwoods'

In [66]:
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L', 
            '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33', 
            20190530, 
            40.8765, 
            -73.9106, 
            500, 
            100)

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf8fe0add57970e257223b9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

In [68]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [69]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Dunkin',Donut Shop,40.877136,-73.906666
4,Starbucks,Coffee Shop,40.877531,-73.905582


In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'AH3Q5ASPPY0MRPKV5F13R3JL1UO1HMYFZFZB4ZQZCGPEWU2L', 
            '1DPVMQDGWBIAXBBVHQ2QZHAUCCQKVFUJYTUPOAZWKSFBJF33', 
            20190530, 
            40.8765, 
            -73.9106, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [94]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=df_geo['Latitude'],
                                   longitudes=df_geo['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights
Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park
Don Mills South
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Fairview
Henry Farm
Oriole
Northwood Park
York University
Bayview Village
CFB Toronto
Downsview East
Silver Hills
York Mills
Downsview West
Downsview
North Park
Upwood Park
Humber Summit
Newtonbrook
Willowdale
Downsview Central
Bedford Park
Lawrence Manor East
Emery
Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [95]:
print(north_york_venues.shape)
north_york_venues.head()

(988, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Arturo's,40.874412,-73.910271,Pizza Place
1,Parkwoods,43.806686,-79.194353,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Parkwoods,43.806686,-79.194353,Tibbett Diner,40.880404,-73.908937,Diner
3,Parkwoods,43.806686,-79.194353,Dunkin',40.877136,-73.906666,Donut Shop
4,Parkwoods,43.806686,-79.194353,Starbucks,40.877531,-73.905582,Coffee Shop


In [96]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Manor,26,26,26,26,26,26
Bayview Village,26,26,26,26,26,26
Bedford Park,26,26,26,26,26,26
CFB Toronto,26,26,26,26,26,26
Don Mills North,26,26,26,26,26,26
Don Mills South,26,26,26,26,26,26
Downsview,26,26,26,26,26,26
Downsview Central,26,26,26,26,26,26
Downsview East,26,26,26,26,26,26


In [97]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 23 uniques categories.


In [100]:
toronto_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supermarket,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Parkwoods,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
toronto_onehot.shape

(988, 24)

In [102]:
north_york_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,American Restaurant,Bakery,Bank,Clothing Store,Coffee Shop,Deli / Bodega,Department Store,Diner,Discount Store,Donut Shop,Gym,Ice Cream Shop,Kids Store,Pharmacy,Pizza Place,Sandwich Place,Seafood Restaurant,Steakhouse,Supermarket,Supplement Shop,Tennis Stadium,Video Game Store,Yoga Studio
0,Bathurst Manor,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
1,Bayview Village,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
2,Bedford Park,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
3,CFB Toronto,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
4,Don Mills North,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
5,Don Mills South,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
6,Downsview,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
7,Downsview Central,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
8,Downsview East,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
9,Downsview North,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.076923,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462


In [104]:
north_york_grouped.shape

(38, 24)

In [105]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Bayview Village----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Bedford Park----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----CFB Toronto----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Don Mills North----
                 venue  freq
0          Coffee Shop  0.08
1       Discount Store  0.08
2       Sandwich Place  0.08
3  American Restaurant  0.04
4             Pharmacy  0.04


----Don 

In [106]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
1,Bayview Village,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
2,Bedford Park,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
3,CFB Toronto,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store
4,Don Mills North,Coffee Shop,Sandwich Place,Discount Store,Yoga Studio,Gym,Bakery,Bank,Clothing Store,Deli / Bodega,Department Store


In [108]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [136]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'CLbat', kmeans.labels_)

north_merged = north_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M6A,North York,Lawrence Heights
3,M6A,North York,Lawrence Manor
4,M3B,North York,Don Mills North


In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_geo['Latitude'], df_geo['Longitude'], north_york_data['Borough'], north_york_data['Neighbourhood']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters